
# `skrub` - less wrangling, more machine learning

## Help in the exploration of the data

In [ ]:
from skrub.datasets import fetch_employee_salaries

employee_salaries = fetch_employee_salaries()
X, y = employee_salaries.X, employee_salaries.y
X.head()

In [ ]:
y.head()

In [ ]:
from skrub import TableReport

table = TableReport(employee_salaries.employee_salaries)
table

In [ ]:
from skrub import patch_display

patch_display()  # you can use skrub.unpatch_display() to disable the display

In [ ]:
X


## Help at preprocessing data

`skrub` comes with a set of additional encoders.

In [ ]:
from skrub import DatetimeEncoder, ToDatetime
from sklearn.pipeline import make_pipeline

encoder = make_pipeline(ToDatetime(), DatetimeEncoder())
encoder.fit_transform(X["date_first_hired"])

In [ ]:
from skrub import TextEncoder

encoder = TextEncoder()
encoder.fit_transform(X["employee_position_title"])

In [ ]:
from skrub import MinHashEncoder

encoder = MinHashEncoder()
encoder.fit_transform(X["employee_position_title"])


`TableVectorizer` helps at reducing the boilerplate of `ColumnTransformer`.

In [ ]:
from skrub import TableVectorizer

vectorizer = TableVectorizer()
vectorizer


`tabular_learner` to help at getting meaningful baselines quickly.

In [ ]:
from skrub import tabular_learner
from sklearn.linear_model import RidgeCV

model = tabular_learner(RidgeCV())
model

In [ ]:
model = tabular_learner("regressor")
model

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, X, y)
cv_results = pd.DataFrame(cv_results)

In [ ]:
cv_results


Table joiner compatible with the scikit-learn API. It allows to handle properly
states between training and prediction for which data wrangling with pure pandas or
polars would fail.

In [ ]:
from skrub import Joiner

airports = pd.DataFrame(
    {
        "airport_id": [1, 2],
        "airport_name": ["Charles de Gaulle", "Aeroporto Leonardo da Vinci"],
        "city": ["Paris", "Roma"],
    }
)
# notice the "Rome" instead of "Roma"
capitals = pd.DataFrame(
    {"capital": ["Berlin", "Paris", "Rome"], "country": ["Germany", "France", "Italy"]}
)
joiner = Joiner(
    capitals,
    main_key="city",
    aux_key="capital",
    max_dist=0.8,
    add_match_info=False,
)
joiner.fit_transform(airports)


## Conclusions

**Vision**
- Less wrangling, more machine learning
- Bring the world of database closer to machine learning

**Wrap-up**
- Additional components to assemble, encode, and vectorize data
- Reduce boilerplate code to get good baseline
- Broader the scope of scikit-learn pipeline to the database world

**Bold vision**
- scikit-learn should be the machine learning toolbox with its numerical
  optimization roots and expertise
- skrub could be where the data preparation happen with integration with
  dataframe-like libraries